In [2]:
using Plots
using Suppressor
using WebIO
plot();
Animation();

┌ Info: Precompiling Suppressor [fd094767-a336-5f1f-9728-57cf17d0bbfb]
└ @ Base loading.jl:1273
┌ Info: Precompiling WebIO [0f1e0344-ec1d-5b48-a673-e5cf874b6c29]
└ @ Base loading.jl:1273


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2277099789129251527\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15205505877000416065\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-2277099789129251527\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [23]:
ks = [100, 1000, 10000]
rs = 0:0.25:3
N0 = 1.
gens = 30
for r in rs
    p = plot(title = "r = $(r)", xlabel = "T", ylabel="N_T_")
    for k in ks    
        Ns = [N0]
        for gen in 1:gens
            Nt = Ns[end]
            push!(Ns, floor(Nt*exp(r*(1-Nt/k))))
        end
        plot!(p, Ns, label="k=$(k)", xaxis=:log, yaxis = :log)
    end
    savefig(p, "popeco/r=$(r)")
    # display(p)
end


In [35]:
ppi_anim = Animation()
p = plot([0],[0], title="r vs Ne", label="")
k = 1000
rs = 0.1:0.025:4.2
N0 = 1.
gens = 3000

@inbounds begin
    for r in rs
        # print(r,",")
        Ns = @fastmath life(k, r, gens)
        plot!(p, collect(fill(r,101)), Ns[end-100:end], st=[:scatter], label="", markersize=2)
        # savefig("popeco\\rvsNe_r=$(r)")
        frame(ppi_anim, p)
    end
end
savefig("ricker")

In [34]:
gif(ppi_anim, "run.gif", fps=10)
display(Node(:img, src = "run.gif?modified=$(rand())"))

(img { src="run.gif?modified=0.28701444080215444" })

┌ Info: Saved animation to 
│   fn = C:\Users\dhruv\Programs\Julia\Lab stuff\run.gif
└ @ Plots C:\Users\dhruv\.julia\packages\Plots\XpHkc\src\animation.jl:98


In [31]:
function life(k, r, gens)
    Ns = [N0]
    for gen in 1:gens
        Nt = Ns[end]
        push!(Ns, Nt*exp(r*(1-Nt/k)))
    end
    return Ns
end
function life_thread(k, r, gens)
    Ns = [N0]
    Threads.@threads for gen in 1:gens
        Nt = Ns[end]
        push!(Ns, Nt*exp(r*(1-Nt/k)))
    end
    return Ns
end

life_thread (generic function with 1 method)

In [49]:
@time life(1000, 1.3)

  0.001482 seconds (34.48 k allocations: 650.141 KiB)


In [53]:
@time @fastmath life_thread(1000,1.3)

  0.000050 seconds (21 allocations: 64.813 KiB)


In [54]:
@time @fastmath life(1000, 1.3)

  0.002236 seconds (34.48 k allocations: 650.141 KiB)
